# Lab 3: Ice Stream Margin
---

Today we will start another two-part lab, this time working on ice stream velocity profile models. Lab 3 will be mostly on the forward model, while Lab 4 will be on the inverse model.  

**Note**: Due to the pace of lectures this year, we have removed a derivation question from this lab. The omitted question will be added to a written assignment instead. As a result, this lab will be shorter than others. 

## Submission

Please save this file (with your additional answer cells included — make sure they run!), download it (**File $\rightarrow$ Download**), and upload to Canvas with your name included in the file name (e.g. lab_03_*name*.ipynb).

## 1. Ice Stream Theoretical Velocities

Soon in lecture, we will determine the velocity of a parallel-sided slab of material flowing down an inclined slope based on the conservation of momentum and a linearly viscous material. The lecture will also cover some other examples of different geometries. One of those will be the flow of material in a square channel inclined at an angle $\alpha$, similar to the diagram below:

<img src="https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_03/icestream.png" width=500/>
<div style="text-align:right"><em>Image courtesy Erin Pettit, OSU</em></div>

We can use this simple geometry to describe an ice stream in Antarctica. Ice streams are fast flowing regions of ice in between slow flowing ones. They have strongly crevassed edges — the "shear margins" — because there is a large amount of shear strain happening there (you estimated a strain rate tensor for these margins in class as well). The first photo below shows a map view of the velocities measured by InSAR, the second photo below shows an photo of a crevassed ice stream margin from an airplane.

<img src="https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_03/Ice_Stream_B.jpg" width=500/>
<img src="https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_03/stream_recovery_shuman.jpg" width=500/>
<div style="text-align:right"><em>Images courtesy Erin Pettit, OSU</em></div>

Ice streams are not typically bounded by bedrock topography (i.e., they don't flow in a bedrock "valley"). They appear due to positive feedbacks in the system that enhance small variations in the strength of the sediment or water availability at the ice-bed interface. But, honestly, it is often difficult to explain exactly why an ice stream appears.

The biggest difference between our calculation in class based on the geometry above and the real ice stream is that ice is **not** a linearly viscous material. Ice has a viscosity that depends on the deviatoric tensor applied:

$$\dot{\epsilon}_{ij} = A(T)\tau^{n-1}_\mathrm{eff}\tau_{ij}\,. \tag{1}$$

In this equation, $\dot{\epsilon}_{ij}$ is the strain rate tensor, $\tau_{ij}$ is the deviatoric stress tensor, and $\tau_\mathrm{eff}$ is the square root of the second invariant of the deviatoric stress tensor ($\sqrt{I_2}$). $n$ is an exponent describing the sensitivity of the deformation to the deviatoric stress tensor. Most studies have suggested that a value of $n = 3$ works best for ice. $A(T)$ is called the "softness parameter," it is a coefficient that depends on temperature $T$ (mostly), orientations of crystals (next most important), impurity content in the ice, and other variations from a simple description of ice rheology. In reality, $A$ can and does vary spatially. **Note that the units of $A$ depend on the value of $n$ chosen.** For $n = 3$, there are suggested values of $A$ published for each temperature for clean Holocene age ice (in Cuffey and Paterson, 2010). **For other values of $n$ there will be different values of $A$, this is primarily because the $n$ determines the units of the equation (stress squared or stress cubed have different units!).**

For this exercise, you will estimate a value of $n$ and its corresponding $A$ that best fits measurements of flow on an ice stream. You can assume that the ice is "isothermal," which means that the ice is all the same temperature. Because this is the biggest influence on $A$, we can assume that $A$ is spatially uniform.

### 1.1 Downslope velocity as a function of distance.

In the omitted derivation question, we would have arrived at this equation representing downslope velocity as a function of distance across the ice stream:

$$\dot{u}_1(x_3)=\frac{2A}{n+1}(\rho g \sin \alpha ) ^n\left[(W/2)^{n+1}-{x_3^{n+1}}\right], \tag{2}$$

where $W$ is the width of the ice stream, $g$ is the gravitational acceleration constant, and $\rho$ is the ice density.

## 2. The Forward Problem

A team of glaciologists led by UAF's late Dr. Keith Echelmeyer measured the position of several stakes in a line across the shear margin of Ice Stream B in West Antarctica. They returned the following year and remeasured the stake locations, determining the velocity and direction of motion at each of the stakes. See the figure below.

<img src="https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_03/data.png" width=700/>
<div style="text-align:right"><em>Image courtesy Erin Pettit, OSU</em></div>

The left ($x_3 = 0$) side of this plot is the centerline of the ice stream. These data are stored in `isbdata.mat`. The average slope (rise/run) of the ice is 0.013.

**Your task is to take their observations and figure out if the common assumption that $n = 3$ works for this situation. If so, what is the best choice of an $A$ parameter. If not, what combination of $n$ and $A$ might work better.** You will do this by using a relatively simple curve fitting process with a grid search over values of $n$ and $A$. This is a simple version of a geophysical *inverse problem*, which are very common in geophysics. An inverse problem means that we are given the answer (the observations), but we need to figure out what the question was (here: the best theoretical model to predict the ice flow). We are trying to infer values for the model parameters from the data. To solve an inverse problem, you need to have the *forward problem* defined. The forward problem states what your observations (data) should be if you pick certain model parameter values. Your forward problem is Equation 2 — given the forward problem and values for $A$ and $n$, width $W$ of the ice stream, and  density $\rho$ of the ice, you can determine the speed of the ice for that particular setting.

Our inverse problem is an *overdetermined* problem. There are only two unknown model parameters ($A$ and $n$) but we have well over 20 data points, which means 20 versions of Equation 2. You *could* calculate a different $A$ and $n$ for any two data points; that would be an exactly determined problem. But how do we decide which two points to use? And wouldn't it be better to use all of our observations, rather than just picking out two? Furthermore, there will be some error in our measurements (as with all measurements), it would also be nice to take that into account in deciding what the best $A$ and $n$ are.

In order to use all of our observations we have to come up with a function that will allow us to compare a predicted $\dot{u}_1^\mathrm{pred} (x_3)$ to the observed  $\dot{u}_1^\mathrm{obs} (x_3)$, given some uncertainty in our measurements, $\delta$.

For the moment, get started on the forward problem:

### 2.1 Write Forward Problem as a Function

First, write two functions using Equation 2 for velocity as a function of distance above. 

* The first will be to calculate $\dot{u_1}=f(\rho,g,\alpha,n,W,A,x_3)$. This will be the main function that you will use. 

* The second will use the same equation just rearranged to calculate a value for $A$ given the value of $\dot{u}_1$ for a given position across the ice stream $x_3$: $A_{\text{estimate}}=f(\rho,g,\alpha,n,W,x_3 , \dot{u}_1)$. You will need this version in order to estimate an $A$ value to be a starting point for your grid search.

In [ ]:
# Write your two functions here!

### 2.2 Plot the observations

Plot the data in `isbdata.mat` in a code cell following the one below (which is reading in the data).

In [ ]:
from scipy.io import loadmat

!curl -O https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_03/isbdata.mat
    
data = loadmat('isbdata.mat', squeeze_me=True)

x3 = data['dpx'] * 1000  # [m]
u1 = data['dpv']  # [m/yr]

### 2.3 Calculate Estimated Profiles for $n = 3$

Estimate an $A$ value for $n=3$ based on just one data point of $\dot{u}_1(x_3)$ and then calculate the velocity for all values of $x_3$. Plot this estimated velocity profile on top of your data. This will help you check that your function is written correctly. 

Would this $A$ estimate be different if you chose a different data point for calculating the $A$ value? Plot two additional velocity profiles (using different data points and therefore different $A$ values) on top of your data to visualize the differences, and add an appropriate legend to your plot. How do the velocity profiles differ? How do these $A$ values compare to those recommended by Cuffey and Paterson? Here is their chart of recommended values:

<img src="https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_03/A_patterson.jpg" width=300/>
<div style="text-align:right"><em>Image courtesy Erin Pettit, OSU</em></div>

Answer in a new code cell below: